In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import glob
from cftime import num2date
from xarray.coding.cftimeindex import CFTimeIndex
import cftime


Get Global CRUJRA & GIMMS LAI 4G data for LSTM training 

In [4]:
# Define a preprocessing function to convert 6-hourly to daily by taking the mean
def pp_daily_mean(ds):
    ds= ds[['TBOT']]
    daily = ds.resample(time='1D').mean()
    return daily

# convert to daily by taking the sum
def pp_daily_sum(ds):
    daily = ds.resample(time='1D').sum()
    return daily

# Directory 
data_dir = '/glade/campaign/cesm/cesmdata/cseg/inputdata/atm/datm7/atm_forcing.datm7.CRUJRA.0.5d.c20241231/three_stream/'
# Temperature, Pressure, Winds, Humidity, and Downward Longwave Radiation
pattern_tpqwl = f'{data_dir}clmforc.CRUJRAv2.5_0.5x0.5.TPQWL.*.nc'

# Open and preprocess all TPQWL files directly to daily
ds_tpqwl_daily = xr.open_mfdataset(
    sorted(glob.glob(pattern_tpqwl)),
    combine='by_coords',
    preprocess=pp_daily_mean
)

In [8]:
ds_temp_8220 = ds_tpqwl_daily.sel(time=slice('1982-01-01', '2020-12-16'))

In [9]:
ds_temp_8220

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 720, time: 14220)
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * time     (time) object 1982-01-01 00:00:00 ... 2020-12-16 00:00:00
Data variables:
    TBOT     (time, lat, lon) float32 dask.array<shape=(14220, 360, 720), chunksize=(1, 360, 720)>

In [10]:
# Precipitation 
pattern_prec = f'{data_dir}clmforc.CRUJRAv2.5_0.5x0.5.Prec.*.nc'

ds_prec_daily = xr.open_mfdataset(
    sorted(glob.glob(pattern_prec)),
    combine='by_coords',
    preprocess=pp_daily_sum
)

ds_prec_8220 = ds_prec_daily.sel(time=slice('1982-01-01', '2020-12-16'))

pattern_solr = f'{data_dir}clmforc.CRUJRAv2.5_0.5x0.5.Solr.*.nc'

ds_solr_daily = xr.open_mfdataset(
    sorted(glob.glob(pattern_solr)),
    combine='by_coords',
    preprocess=pp_daily_sum
)

ds_solr_8220 = ds_solr_daily.sel(time=slice('1982-01-01', '2020-12-16'))
